In [22]:
Vertices=[ProcessGameState.Point(-1735, 250), ProcessGameState.Point(-2024, 398), ProcessGameState.Point(-2806, 742), ProcessGameState.Point(-2472, 1233), ProcessGameState.Point(-1565, 580)]
Rounds=set()
for index, row in Team2onT.iterrows():
  PoI=[row['x'],row['y'],row['z']]
  if ProcessGameState.BoundaryCheck(Vertices,PoI):
    Rounds.add(row['round_num'])
  
print(Rounds)

print(Team2onT.round_num.unique())

{16}
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30]


In [13]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
from google.colab import drive
drive.mount('/content/drive')
curr_dir=os.getcwd()
parquet_file='/content/drive/MyDrive/Software Engineer Assessment/data/game_state_frame_data.parquet'
 
class ProcessGameState:
  class Point:
      def __init__(self, x, y):
          self.x = x
          self.y = y
  
  class line:
      def __init__(self, p1, p2):
          self.p1 = p1
          self.p2 = p2
  def __init__(self, parquet_file):
      self.MatchData=pd.read_parquet(parquet_file,engine='auto') 
      self.Team2onT=self.MatchData.loc[(self.MatchData['team'] == 'Team2')&(self.MatchData['side'] == 'T') ]
      self.T2onTSiteB=self.Team2onT.loc[(self.Team2onT['area_name']=='BombsiteB')&(self.Team2onT['bomb_planted']==False)]\
      [['player', 'clock_time', 'round_num', 'inventory']].sort_values(['round_num', 'clock_time'], ascending=[True, False])
      self.Team2onCT=self.MatchData.loc[(self.MatchData['team'] == 'Team2')&(self.MatchData['side'] == 'CT') ]
      self.T2onCTSiteB=self.Team2onCT.loc[(self.Team2onCT['area_name']=='BombsiteB')&(self.Team2onCT['bomb_planted']==False)] [['player', 'clock_time', 'round_num', 'x', 'y']]
  def onLine(l1, p):
      # Check whether p is on the line or not
      if (
          p.x <= max(l1.p1.x, l1.p2.x)
          and p.x <= min(l1.p1.x, l1.p2.x)
          and (p.y <= max(l1.p1.y, l1.p2.y) and p.y <= min(l1.p1.y, l1.p2.y))
      ):
          return True
      return False
  
  def direction(a, b, c):
      val = (b.y - a.y) * (c.x - b.x) - (b.x - a.x) * (c.y - b.y)
      if val == 0:
          # Collinear
          return 0
      elif val < 0:
          # Anti-clockwise direction
          return 2
      # Clockwise direction
      return 1
  
  def isIntersect(l1, l2):
      # Four direction for two lines and points of other line
      dir1 = ProcessGameState.direction(l1.p1, l1.p2, l2.p1)
      dir2 = ProcessGameState.direction(l1.p1, l1.p2, l2.p2)
      dir3 = ProcessGameState.direction(l2.p1, l2.p2, l1.p1)
      dir4 = ProcessGameState.direction(l2.p1, l2.p2, l1.p2)
  
      # When intersecting
      if dir1 != dir2 and dir3 != dir4:
          return True
  
      # When p2 of line2 are on the line1
      if dir1 == 0 and ProcessGameState.onLine(l1, l2.p1):
          return True
  
      # When p1 of line2 are on the line1
      if dir2 == 0 and ProcessGameState.onLine(l1, l2.p2):
          return True
  
      # When p2 of line1 are on the line2
      if dir3 == 0 and ProcessGameState.onLine(l2, l1.p1):
          return True
  
      # When p1 of line1 are on the line2
      if dir4 == 0 and ProcessGameState.onLine(l2, l1.p2):
          return True
  
      return False
  
  def is_within_polygon(poly, p):
      # When polygon has less than 3 edge, it is not polygon
      n=len(poly)
      if n < 3:
          return False
  
      # Create a point at infinity, y is same as point p
      exline = ProcessGameState.line(p, ProcessGameState.Point(9999, p.y))
      count = 0
      i = 0
      while True:
          # Forming a line from two consecutive points of poly
          side = ProcessGameState.line(poly[i], poly[(i + 1) % n])
          if ProcessGameState.isIntersect(side, exline):
              # If side is intersects ex
              if (ProcessGameState.direction(side.p1, p, side.p2) == 0):
                  return ProcessGameState.onLine(side, p);
              count += 1
          
          i = (i + 1) % n;
          if i == 0:
              break
  
      # When count is odd
      return count & 1;
  def BoundaryCheck(Vertices, PointOfInterest, ZBound=[285, 421]):
    if PointOfInterest[2]<ZBound[0] or PointOfInterest[2]>ZBound[1]:
      return False
    if len(Vertices)>3:
      return ProcessGameState.is_within_polygon(Vertices,ProcessGameState.Point(PointOfInterest[0],PointOfInterest[1]))
    else:
      return False
  def getWeapons(dfRow):
    WeaponClasses=set()
    if dfRow.inventory is not None:
      for weapon in dfRow.inventory:
        WeaponClasses.add(weapon['weapon_class'])
    return WeaponClasses
  def col_to_numpy(df, col):
    return df[col].to_numpy()
 
# Driver code
#polygon = [  Point( 0, 0 ), Point( 1, -1 ),  Point( 2, -0.5),Point( 2.667, 0.667 ), Point( 2, 2 ) ];
#p = Point( 2, -0.4);
#n=5

polygon = [  ProcessGameState.Point( 0, 0 ), ProcessGameState.Point( 1, 0 ),  ProcessGameState.Point( 1, 1),ProcessGameState.Point( 0,1 )];
p = ProcessGameState.Point( 0.5, 0.5 );
n=4
# Function call
if (ProcessGameState.is_within_polygon(polygon, p)):
    print("Point is inside.")
else:
    print("Point is outside.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Point is inside.


In [14]:
Vertices=[ProcessGameState.Point(-1735, 250), ProcessGameState.Point(-2024, 398), ProcessGameState.Point(-2806, 742), ProcessGameState.Point(-2472, 1233), ProcessGameState.Point(-1565, 580)]
Rounds=set()
ZBound=[285, 421]
PGS=ProcessGameState(parquet_file)
for index, row in PGS.Team2onT.iterrows():
  PoI=[row['x'],row['y'],row['z']]
  if ProcessGameState.BoundaryCheck(Vertices,PoI,ZBound):
    Rounds.add(row['round_num'])
TRounds=len(PGS.Team2onT.round_num.unique())

print("Out of a total of {} T Rounds from Team 2 , they entered light blue boundary in Round(s) {}".format(TRounds,Rounds))
print("And since they entered the boundary only in Rounds 16, which is a pistol rounds, this is not a common strategy that Team 2 on T tends to deploy")

Out of a total of 15 T Rounds from Team 2 , they entered light blue boundary in Round(s) {16}
And since they entered the boundary only in Rounds 16, which is a pistol rounds, this is not a common strategy that Team 2 on T tends to deploy


In [8]:
def is_within_polygon(polygon, point):
    A = []
    B = []
    C = []  
    for i in range(len(polygon)):
        p1 = polygon[i]
        p2 = polygon[(i + 1) % len(polygon)]
        
        # calculate A, B and C
        a = -(p2.y - p1.y)
        b = p2.x - p1.x
        c = -(a * p1.x + b * p1.y)

        A.append(a)
        B.append(b)
        C.append(c)

    D = []
    for i in range(len(A)):
        d = A[i] * point.x + B[i] * point.y + C[i]
        D.append(d)

    t1 = all(d >= 0 for d in D)
    t2 = all(d <= 0 for d in D)
    return t1 or t2

if __name__ == '__main__':
    polygon = [  Point( 0, 0 ), Point( 1, -1 ),  Point( 2, -0.5),Point( 2.667, 0.667 ), Point( 2, 2 ) ];
    p1 = Point(2, -0.5)
    print(is_within_polygon(polygon, p1)) # returns True
    p2 = Point(1, 1)
    print(is_within_polygon(polygon, p2)) # returns False

True
True


# New Section